# Welcome!

Welcome to the HueDX API sample notebook. Although not a holistic dive into all the possiblities the HueDX platform has to offer, the following code blocks should be enough for any data science team to immediately begin using all core functionality related to the SignalDetector and HueCard processing.

### For Additional API Details
- Production Environment: https://hue-api.com/prod
- Test Environment: https://hue-api.com/test
- Documentation: https://docs.hue-api.com

## Step 1 - Common Imports, Variables and Utilities

In order to interact with the API and its payloads, the following standard imports are recommended, but not required. You may replace any functionality with internal tools or third party libraries without affecting the API's results.

In [ ]:
import json
import time
import requests


def get_job_results(job_id):
    payload={"jobId": job_id}
    response = requests.request("GET", job_url, headers=headers, params=payload)

    while response.json()[0]["status"] != "success":
        if response.json()[0]["status"] == "failed":
            print("Job failed!")
            return response
        time.sleep(2)
        print("Waiting for job to complete...")
        response = requests.request("GET", job_url, headers=headers, params=payload)

    return requests.request("GET", results_url, headers=headers, params=payload)


upload_url = "https://hue-api.com/prod/upload/generate-url"
job_url = "https://hue-api.com/prod/jobs"
results_url = "https://hue-api.com/prod/results"

## Step 2 - Obtain Your Credentials

Use the `login` API to obtain your credentials which will need to be passed to subsequent API calls. In the following example the user's information is stored in a file called `creds.json` and contains the username, password and appToken for the HueAPI.

In [ ]:
# Retrieve the credentials for obtaining API authorization
with open("creds.json") as f:
    creds = json.load(f)

#Configure the necessary credentials
email = creds['email']
password = creds['password']
app_token = creds['appToken']
    
header = {
    "appToken": app_token,
    "authorizationToken": "0" # Required field, but can be left as 0 here
}

# Obtain the authorization token needed for other API calls
login_url = "https://hue-api.com/prod/login"
response = requests.post(login_url, json={"email": email, "password": password}, headers=header)
authorization_token = response.json()["token"]

## Step 3 - Uploading Your Image

In order to upload your image, you must first retrieve a pre-signed URL using the `generate-url` API. The returned URL will then be used to `PUT` your image into cloud storage for processing.


In [ ]:
payload = {
    "format": "png", 
    "cameraMake": "Kyocera", # optional
    "cameraModel": "VP-210", # optional
    "cameraOSVersion": "1.0", # optional
    "tag": "NewAssay", # optional
    "experimentName": "110000 Pixel Camera Test" # optional
    }

headers = {"appToken": app_token, "authorizationToken": authorization_token}

response = requests.request("POST", upload_url, headers=headers, json=payload)
upload_loc = response.json()

In [ ]:
filename = './assets/sample.png'

with open(filename, "rb") as upload_file:
    files = {'file': (filename, upload_file)}
    upload_response = requests.post(upload_loc['url'], data=upload_loc['fields'], files=files)


# Steps 4-6 : Utilizing the Colorimetric Services

Currently, the HueAPI exposes 3 primary colorimetric services that data science teams may find useful:

- Colorimetric Analysis: This service provides basic analysis of a color swatch and generates independent, quantitative measures of the image.
- HueCard Processing: This service must be combined with sample images taken using a HueCard. The HueCard processing service is responsible for registering the image (i.e. skew/warp/orientation correction), applying color correction to account for lighting differences, and extracting the ROI reaction windows from the card image.
- Signal Detection: This service provides advanced analytics of color swatches to create multivariate, quantitative and visualized analyses of an experiment to determine experimental quality and success.

## Colorimetric Analysis

In [ ]:

payload = {
    "service": "colorimetricAnalysis",
    "rawImageS3Key": upload_loc["fields"]["key"],
    "tag": "ColorAnalysis", # optional
    "experimentName": "RedAnalysis", # optional
    "extractRoi": {
        "origin": {
            "x": 250,
            "y": 250
        },
        "dimensions": {
            "width": 90,
            "height": 90
        }
    } 
}

response = requests.request("POST", job_url, headers=headers, json=payload)
job_id = response.json()["jobId"]

In [ ]:
payload={"jobId": job_id}
response = requests.request("GET", job_url, headers=headers, params=payload)

while response.json()[0]["status"] != "success":
    if response.json()[0]["status"] == "failed":
        print("Job failed!")
        break
    time.sleep(2)
    print("Waiting for job to complete...")
    response = requests.request("GET", job_url, headers=headers, params=payload)


colorimetric_results = get_job_results(job_id)

## HueCard Processing

HueCard processing uses the HueDX HueCard to process reaction samples either as a stand alone service that exposes the final corrected image or as a pipeline that uses the corrected image to run each reaction sample through a quantitative model and produce inference results. The type of processing is handled by the `invocationType` field which may be either `stand-alone` or `pipeline`. By default, the `cardProcessor` service applies color correction to the card after image registration. To toggle this on or off as a step, use the `colorCorrection` attribute.

The `selectedConfigurations` object helps keep track of each reaction ROI which is automatically extracted from the image provided in `imageS3Key`.  Within the panel configurations is a `modelId` attribute. This parameter allows for easy and rapid selection of different models for the purposes of producing quantitative measurements of your assays. After a model has been developed that produces satisfactory and performant results for your assays, that modelId is considered static. The model selection for your production HueCard configuration is generally managed, chosen, and locked by your HueDx administrator.

In [ ]:
payload = {
    "service":"cardProcessor",
    "imageS3Key": upload_loc["fields"]["key"],
    "colorCorrection": True,
    "deviceTypeId": "HUE-LVR-1-1",
    "context": {
        "invocationType": "stand-alone"
    },
    "selectedConfigurations": {
        "panel1": {
            "configurationName": "blood",
            "modelId": "bloodModelId"
        },
        "panel2": {
            "configurationName": "plasma",
            "modelId": "plasmaModelId"
        },
        "panel3":{
            "configurationName": "blood"
        },
        "panel4": {
            "configurationName": "blood"
        },
        "panel5": {
            "configurationName": "blood"
        },
        "panel6": {
            "configurationName": "blood"
        }
    }
}

response = requests.request("POST", job_url, headers=headers, json=payload)
job_id = response.json()["jobId"]

card_processor_results = get_job_results(job_id)

In [ ]:
card_processor_results.json()

## Signal Detection

In [ ]:
sample_loc = card_processor_results.json()[0]["resultData"]["panels"][0]["s3Key"]
experiment_images = []
for fake_concentration in range(1, 15):
    experiment_images.append({"s3Key": sample_loc, "concentration": fake_concentration})

payload = {
    "service": "signalDetector",
    "useWeights": True,
    "criticalRange": (4., 8.),
    "images": experiment_images,
    "backgroundImageS3Key": sample_loc
}

response = requests.request("POST", job_url, headers=headers, json=payload)
job_id = response.json()["jobId"]

signal_results = get_job_results(job_id)

In [ ]:
signal_results.json()

## Step 7 - Retrieving Results

In [ ]:
header = {
        "appToken": creds['appToken'],
        "authorizationToken": loginToken.json()["token"]
    }
payload = {"resultId": resultIds}

return requests.get(url, headers=header, params=payload)